# Evolution of Einstein's equations

# Initial data , setting length of the domain and discretization


# Parameters

In [496]:
m = 3
A = 0.001#55#0.05#0.049375#0.07#0.05#0.1124921875#0.125
run = 1

1

In [497]:
res=m;
N=2.0^m*200.0/2.0
Xf=Float128(1.0)#1.0;

dx=Float128(Xf/N);#Xf/N
dt=dx#round(dx,digits=10);#dx
Nt=2.0^m*200.0/2.0
Tf=Nt*dt; #final time
#print(Tf)

In [498]:
print(N)

800.0

In [499]:
println("dx =", dx)
println("dt =", dt)
println("dx/dt =", dx/dt)

dx =1.25000000000000000000000000000000002e-03
dt =1.25000000000000000000000000000000002e-03
dx/dt =1.00000000000000000000000000000000000e+00


In [500]:
global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA"
#global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA"

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA"

# Grid

In [501]:
include("./ScalarField.jl");

In [502]:
ori=Float128(0.0)#0.0;
initX1 = nothing
N=int(N)
#initX1=range(ori, stop=Xf, step=dx);
initX1=create_range(ori,Xf,dx,N)
#initX = range(round(ori-3.0*dx,digits=10), stop=Xf+3.0*dx, step=round(dx,digits=10))
initX=create_range(ori-3.0*dx,Xf+3.0*dx,dx,N+6)

L=length(initX);
println("step size is  ", dx)
println("the time step is ", dt)

step size is  1.25000000000000000000000000000000002e-03
the time step is 1.25000000000000000000000000000000002e-03


# Initial data of m β Ψ and Ψ,x

In [503]:
using Dierckx
using Quadmath

####

#initm=zeros(L)
#initbeta=zeros(L)
#initpsi=zeros(L)
#initderpsi=zeros(L)
initm=Array{Float128}(undef, L)
initbeta=Array{Float128}(undef, L)
initpsi=Array{Float128}(undef, L)
initderpsi=Array{Float128}(undef, L)
#initm=Array{Float64x8}(undef, L)
#initbeta=Array{Float64x8}(undef, L)
#initpsi=Array{Float64x8}(undef, L)
#initderpsi=Array{Float64x8}(undef, L)

state_array=[initm initbeta initpsi initderpsi initX];

#PSI
r0=0.7#0.01#0.7#0.01#0.7#0.7#0.7#0.01#0.7#0.3
sigma=0.3
initpsi[4:L-3] = init_gaussian(initX1,r0,sigma,A)

state_array[:,3] = initpsi
state_array = ghost(state_array)

#PSI,X FROM PSI
initderpsi[4:L-3] = init_gaussian_der(initX1,r0,sigma,A)
state_array[:,4] = initderpsi
state_array=ghost(state_array)

####

derpsi_func = Spline1D(initX[4:L-3], state_array[4:L-3,4],  k=4);

In [521]:
derpsi_func(state_array[:,5])

807-element Vector{Float64}:
  1.0013661530759225e-28
  1.0013661530759225e-28
  1.0013661530759225e-28
  1.0013661530759225e-28
  2.0886892887540083e-11
  8.616805109762858e-11
  1.999535596097289e-10
  3.6660221229479793e-10
  5.907338974067079e-10
  8.772425322121595e-10
  1.2313095691420101e-9
  1.6584180966935427e-9
  2.164367559189279e-9
  ⋮
  1.0805021434729487e-138
  3.396660502629746e-139
  6.060193412465916e-140
 -7.544489769240383e-141
  7.001204500531497e-140
  3.287388408961129e-140
  7.872511063555182e-141
  0.0
  0.0
  0.0
  0.0
  0.0

In [504]:
y0=[0 0 0]
state_array[4:L-3,1:3] = twod_n_rk4wrapper(RHS,y0,initX[4:L-3],0,derpsi_func,state_array[:,:]);

state_array = ghost(state_array);

In [505]:
"""using Plots #showing initial data and the initial status of the horizon finder
plot(initX[4:L-3], state_array[4:L-3,1:4], xaxis="x",title="Initial Data", labels=["m" "beta" "psi" "psi,x"])"""

"using Plots #showing initial data and the initial status of the horizon finder\nplot(initX[4:L-3], state_array[4:L-3,1:4], xaxis=\"x\",title=\"Initial Data\", labels=[\"m\" \"beta\" \"psi\" \"psi,x\"])"

# Defining the initial state array for the evolution m β Ψ and Ψ,x

In [506]:
using CSV, Tables
#CSV.write(dir*"/time_step0.csv", Tables.table(transpose(Matrix(state_array))), writeheader=false)
CSV.write(dir*"/res$res/time_step0.csv", Tables.table(state_array), writeheader=false)

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3/time_step0.csv"

In [507]:
time=0.0
criticality=0.0
explode=0.0
evol_stats = [criticality A sigma r0 time explode run]
monitor_ratio = zeros(L)
CSV.write(dir*"/parameters.csv", Tables.table(evol_stats), writeheader=true, header=["criticality", "A", "sigma", "r0", "time", "explode", "run"])

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv"

In [508]:
"""ginit=dt_scale(initX,state_array[:,1],state_array[:,2],dx)
println(update_dt(initX,state_array[:,1],state_array[:,2],dx,ginit)/dt)"""

"ginit=dt_scale(initX,state_array[:,1],state_array[:,2],dx)\nprintln(update_dt(initX,state_array[:,1],state_array[:,2],dx,ginit)/dt)"

 ## Time evolution

fixed grid

In [509]:
finaltime=1.3
evol_stats, T_interp = timeevolution(state_array,finaltime,dir*"/res$res",run);
CSV.write(dir*"/timearray$res.csv", Tables.table(T_interp), writeheader=false);
#CSV.write(dir*"/parameters.csv", Tables.table(evol_stats), writeheader=false);

iteration 10 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 1.24999999999999999999999999999999991e-02
iteration 20 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 2.50000000000000000000000000000000042e-02
iteration 30 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 3.75000000000000000000000000000000108e-02
iteration 40 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 5.00000000000000000000000000000000205e-02
iteration 50 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 6.25000000000000000000000000000000241e-02
iteration 60 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 7.50000000000000000000000000000000337e-02
iteration 70 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.75000000000000000000000000000000433e-02
iteration 80 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 1.00000000000000000000000000000000053e-01


iteration 670 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.37499999999999999999999999999989330e-01
iteration 680 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.49999999999999999999999999999989099e-01
iteration 690 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.62499999999999999999999999999988868e-01
iteration 700 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.74999999999999999999999999999988637e-01
iteration 710 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.87499999999999999999999999999988406e-01
iteration 720 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 8.99999999999999999999999999999988175e-01
iteration 730 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 9.12499999999999999999999999999987944e-01
iteration 740 dt is 1.25000000000000000000000000000000002e-03, time of iteration is 9.24999999999999999999999999999987

 ## Movie of the evolution

In [359]:
using Plots, CSV, DataFrames
import GR
anim = @animate for n in 1020:10:3200
    
    
    IJulia.clear_output(true)
    
    functions3=DataFrame(CSV.File(dir*"/res$res/time_step$n.csv",header=false, delim=","))
    plot()
        plot1=functions3.Column1[4:L-3];
        plot2=functions3.Column2[4:L-3];
        plot3=functions3.Column3[4:L-3];
        plot4=functions3.Column4[4:L-3];
        plot5=functions3.Column5[4:L-3];
    temp=round(n*dt,digits=2)
    
    plot!(initX1, [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],title="$temp", ylim=[-0.001, 0.001],xlim=[0.0, 1])#, xlim=[0, 0.1])# |> display 
    #plot!(initX[4:L-3], [plot1], labels=["m" "beta" "psi" "psi,x"],ylim=[-1,1])#, xlim=[0, 0.1]) |> display 
    
    end

gif(anim, "anim_res$res.gif", fps = 1200)

LoadError: ArgumentError: "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res2/time_step1020.csv" is not a valid file or doesn't exist

In [360]:
epsilon(dt,dx)

7.81250000000000000000000000000000000e-03

In [361]:
twod_epsilon(dt,dx)

3.12500000000000000000000000000000000e-02

In [362]:
using Plots, CSV, DataFrames
import GR
anim = @animate for n in 10:10:1000#length(T)-1
    
    IJulia.clear_output(true)
    
    functions3=DataFrame(CSV.File(dir*"/res$res/monitor_ratio$n.csv",header=false, delim=","))
    plot()
        plot1=functions3.Column1[4:L-3];
    
    plot!(initX1, [plot1], labels=["monitor ratio"], ylim=[0, 1])# |> display 
    #plot!(initX[4:L-3], [plot1], labels=["m" "beta" "psi" "psi,x"],ylim=[-1,1])#, xlim=[0, 0.1]) |> display 
    
    end

gif(anim, "anim_res$res.gif", fps = 5)

LoadError: ArgumentError: "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res2/monitor_ratio10.csv" is not a valid file or doesn't exist

In [363]:
epsilon(dt,dx)

7.81250000000000000000000000000000000e-03

In [147]:
twod_epsilon(dt,dx)

0.3125